In [80]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random 
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
import re
%matplotlib inline
import pandas as pd

In [81]:
df=pd.read_csv("reviews.tsv", sep='\t')

In [82]:
# labels = df['rating']
# reviews = df['review_text']

Cleaning data from zero rating scores.

In [83]:
cleaned = df.loc[df['rating'].isin([1, 2, 3, 4, 5])]
print(cleaned)

           permalink  rating  \
0         1000095379     4.0   
1         1000095379     1.0   
2         1000095379     4.0   
3         1000095379     1.0   
4         1000095379     5.0   
...              ...     ...   
328640  245591834558     5.0   
328641  245591834558     5.0   
328642  245591834558     5.0   
328643  245591834558     1.0   
328644  245591834558     1.0   

                                              review_text  
0       Вкусная питца, естт летнее кафе, доставка, при...  
1       Думал можно днём с детьми посидеть на открытой...  
2       Есть веранда, вкусная пицца, делают по половин...  
3                        Уже второй раз забивают на заказ  
4                                     Любимая пиццерия! 👍  
...                                                   ...  
328640                            Очень вкусная пицца!!!!  
328641  Лучший борщ на Роза Хутор!!! Большая порция, т...  
328642                        Все понравилось! Пицца норм  
328643  обслужи

Taking the first 30.000 raviews.

In [137]:
cleaned = cleaned[:50000]
# print(cleaned['rating'])

Cleaning reviews from stop-words, etc.

In [138]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [139]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_russian = stopwords.words('russian')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_russian and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
    
    tweets_clean = " ".join(tweets_clean)

    return tweets_clean

In [142]:
cleaned["new_rev_text"] = cleaned["review_text"]
cleaned["new_rev_text"] = cleaned["new_rev_text"].apply(lambda x: process_tweet(x))
print(cleaned)

        permalink  rating                                        review_text  \
0      1000095379     4.0  Вкусная питца, естт летнее кафе, доставка, при...   
1      1000095379     1.0  Думал можно днём с детьми посидеть на открытой...   
2      1000095379     4.0  Есть веранда, вкусная пицца, делают по половин...   
3      1000095379     1.0                   Уже второй раз забивают на заказ   
4      1000095379     5.0                                Любимая пиццерия! 👍   
...           ...     ...                                                ...   
30044  1118106573     5.0  Потрясающее место! Симбиоз европейского дизайн...   
30045  1118106573     5.0  Вкусно, дорого, стало не так душевно, как было...   
30046  1118106573     5.0  После ремонта внутри просто шикарно! Мягкие кр...   
30047  1118106573     5.0  Потрясающее место! Ходили на новогодних праздн...   
30048  1118106573     5.0  Вкусная еда по приемлимым ценам, уютная обстан...   

                                       

Getting sentiments from rating.

In [141]:
def get_sentiment(n):
    return 1 if n >= 4 else 0

In [128]:
cleaned["sentiment"] = cleaned["rating"].apply(get_sentiment)

In [129]:
print(cleaned)

        permalink  rating                                        review_text  \
0      1000095379     4.0  Вкусная питца, естт летнее кафе, доставка, при...   
1      1000095379     1.0  Думал можно днём с детьми посидеть на открытой...   
2      1000095379     4.0  Есть веранда, вкусная пицца, делают по половин...   
3      1000095379     1.0                   Уже второй раз забивают на заказ   
4      1000095379     5.0                                Любимая пиццерия! 👍   
...           ...     ...                                                ...   
30044  1118106573     5.0  Потрясающее место! Симбиоз европейского дизайн...   
30045  1118106573     5.0  Вкусно, дорого, стало не так душевно, как было...   
30046  1118106573     5.0  После ремонта внутри просто шикарно! Мягкие кр...   
30047  1118106573     5.0  Потрясающее место! Ходили на новогодних праздн...   
30048  1118106573     5.0  Вкусная еда по приемлимым ценам, уютная обстан...   

                                       

Splitting data into test and trained samples.

In [130]:
#X_1, y = np.arange(10).reshape((5, 2)), range(5)
y = cleaned['sentiment']
X = cleaned['new_rev_text']

In [131]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)

Applying logistic regression.

In [132]:
from sklearn.feature_extraction.text import CountVectorizer

In [133]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
trained_x = vectorizer.fit_transform(X_train)
test_x = vectorizer.transform(X_test)

In [134]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [135]:
model = LogisticRegression()
model.fit(trained_x, y_train)
y_pred_class = model.predict(test_x)

In [136]:
accuracy_score(y_test, y_pred_class)

0.9046666666666666